# Basic llama index usage

## install libraries

In [1]:
!pip install llama_index
!pip install langchain

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

## import libraries and set api key for gpt-3

In [58]:
import os
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

os.environ['OPENAI_API_KEY'] = "your key here"

## load data and create index

In [50]:
from llama_index import download_loader
import urllib.request
import xml.etree.ElementTree
import xml.etree.ElementTree as ET

RemoteReader = download_loader("RemoteReader")
loader = RemoteReader()

deepshore_webpage = urllib.request.urlopen('https://deepshore.de/sitemap.xml')
sitemap = ET.fromstring(deepshore_webpage.read())

#for child in sitemap:
#    print(child.tag)
documents = []

for url in sitemap.findall('{http://www.sitemaps.org/schemas/sitemap/0.9}url'):
    location = url.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc').text
    if "https://deepshore.de/knowledge/" in location:
        #print(location)
        try:
            documents = documents + loader.load_data(url=location)
        except:
            print(f'error adding with ${location}')
            pass # ignore errors      

print(len(documents))


error adding with $https://deepshore.de/knowledge/2020-01-06
81


## Create Index from Documents

In [51]:
from llama_index import GPTVectorStoreIndex

index = GPTVectorStoreIndex.from_documents(documents)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 874283 tokens
> [build_index_from_nodes] Total embedding token usage: 874283 tokens


## load index from storage

In [56]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage")
# load index
index = load_index_from_storage(storage_context)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.


## persist index to storage

In [52]:
index.storage_context.persist()

## query index

In [59]:
query_engine = index.as_query_engine()
response = query_engine.query("Was ist das cloud native archive von Deepshore?")

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 12 tokens
> [retrieve] Total embedding token usage: 12 tokens
> [retrieve] Total embedding token usage: 12 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2215 tokens
> [get_response] Total LLM token usage: 2215 tokens
> [get_response] Total LLM token usage: 2215 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


In [70]:
print(response.extra_info)

for ex_i in response.extra_info:
    print(response.extra_info[ex_i]["Source"])

{'2aef3700-a718-49f6-8f00-3545eeb3f5de': {'Source': 'https://deepshore.de/knowledge/2021-11-25'}, '0ccf3a9b-8f24-4c99-bbb2-5e23cdc9b45f': {'Source': 'https://deepshore.de/knowledge/2021-11-25'}}


AttributeError: 'dict' object has no attribute 'Source'